In [3]:
import cv2 
import os

def create_subfolder(i, file_list, overall_dir): 
    # overall_dir = 'equations1'
    subfolder_name = file_list[i-1][:-4]
    #Create the subfolder under 'all_equations' if it doesn't exist
    subfolder_path = "./processed_data/" + overall_dir + "/" + subfolder_name
    #print(subfolder_path)
    if not os.path.exists(subfolder_path):
        os.mkdir(subfolder_path)
        print(f"Folder '{subfolder_name}' created under 'processed_data/{overall_dir}' folder.")


def recognise(image_path, file_list, overall_dir):
    for i in range(1, len(image_path)+1): #(Change to for loop)
        # path(image filename)
        path = image_path[i-1]
        
        # Reading an image in grayscale mode 
        image = cv2.imread(path,0) 
        #print(image.shape)
        
        # Window name in which image is displayed 
        #window_name = 'image'
        
        # Using cv2.imshow() method 
        # Displaying the image 
        #cv2.imshow(window_name, image) 
        
        # waits for user to press any key 
        # (this is necessary to avoid Python kernel form crashing) 
        #cv2.waitKey(0) 
        
        # closing all open windows 
        #cv2.destroyAllWindows()
        
        # --------------------------------------------------------------    #--------------------------------------------------------------------------------------------------
        bb_info_list = []

        min_contour_area = 100
        image_with_rectangles = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print(contours)
        image_with_rectangles = image.copy()
        image_with_rectangles = cv2.cvtColor(image_with_rectangles, cv2.COLOR_GRAY2BGR)
        
        from PIL import Image 
        # get image 
        img = Image.open(path) 
        
        # get width and height 
        width = img.width 
        height = img.height

        for contour in contours:
            contour_area = cv2.contourArea(contour)
            #print(contour_area)
            if contour_area > min_contour_area:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image_with_rectangles, (x, y), (x + w, y + h), (0, 0, 255), 1)
                [xmin, xmax, ymin, ymax,  img_width, img_height] = x, (x+w), y, (y+h), img.width, img.height 
                bb_info_list.append([xmin, xmax, ymin, ymax, img_width, img_height])
                #print([xmin, xmax, ymin, ymax,  img_width, img_height])
        #print(len(bb_info_list))

        #Create a folder with the image filename, save components of image in the folder
        
        #for i in range(1,3): #(To delete when changing code to for loop)
        
        # Path to 'all_equations' folder
        #print(os.path.exists("./" + all_equations_dir))
        if not os.path.exists("./processed_data/" + overall_dir):
            os.mkdir("./processed_data/" + overall_dir)
            print(f'processed_data/{overall_dir} folder created')
            create_subfolder(i, file_list, overall_dir)
        else:
            create_subfolder(i, file_list, overall_dir)
        
        #Cropped images into 
        bb_info_list.sort(key=lambda bbox: bbox[0])
        for j in range(len(bb_info_list)):
            xmin, xmax, ymin, ymax, img_width, img_height = bb_info_list[j] #Each element is a list with 6 elements
            #bb_image = image_with_rectangles[ymin:ymax, xmin:xmax] #Make component image black background
            bb_image = cv2.bitwise_not(image_with_rectangles[ymin:ymax, xmin:xmax]) #Make component image white background
            cv2.imwrite(f'./processed_data/{overall_dir}/{file_list[i-1][:-4]}/Component {j}.jpg', bb_image)
            #cv2.imshow(window_name,image_with_rectangles[ymin:ymax, xmin:xmax])
            #cv2.waitKey(0) 
        
            # closing all open windows 
            #cv2.destroyAllWindows()


In [ ]:
# Replace with the actual folder path that contains the images to pass through bounding box & cropping function
folder_path = './equation_dataset/'
# dir_to_save_all_processed_images (PLEASE CHANGE SO THAT THE FOLDERS WILL NOT BE OVERRIDE)
overall_dir = "equations_test"

# List all files in the folder
file_list = os.listdir(folder_path)

# Filter the list to include only image files (you can adjust the file extensions)
image_extensions = ['.jpg', '.png', '.jpeg']
image_paths = [os.path.join(folder_path, file) for file in file_list if any(file.lower().endswith(ext) for ext in image_extensions)]

recognise(image_paths, file_list, overall_dir)